In [2]:
# !pip install vaderSentiment
# !pip install geopy

In [1]:
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor

from  geopy.geocoders import Nominatim
import reverse_geocoder as rg

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

import re
import csv

import credentials #stored API keys etc

import time

In [2]:
API_key=credentials.API_key
API_secret_key=credentials.API_secret_key
access_token=credentials.access_token
access_token_secret=credentials.access_token_secret

In [3]:
geolocator = Nominatim(user_agent="http")
sentiment = SentimentIntensityAnalyzer() 


#run only the first time
column_headers=['User_Id','Created_at','Country_code','State','Latitude','Longitude','Tweet_text','Sentiment_compound','Overall_sentiment']

'''with open('etweets.csv','w') as file:
    writer=csv.DictWriter(file,fieldnames=column_headers)
    writer.writeheader()'''
    

"with open('etweets.csv','w') as file:\n    writer=csv.DictWriter(file,fieldnames=column_headers)\n    writer.writeheader()"

In [4]:
def deEmojify(text):
    #Removing non-ASCII characters (emoji characters)
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [5]:
class MyStreamListener(tweepy.StreamListener):
    def on_status(self, tweet):
        #Extracting tweets
        
        if tweet.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        user_location = deEmojify(tweet.user.location)
        if user_location!=None:
            try:
                id_str = tweet.id_str
                created_at = tweet.created_at
                
                #Pre-processing the text
                text = deEmojify(tweet.text)    
                #Removing RT, links and special characters in the tweet text
                text=' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet.text).split())
                text=' '.join(re.sub("RT"," ",text).split())
                
                #Getting the co-ordinates of the user
                loc = geolocator.geocode(user_location)
                longitude = loc.longitude
                latitude = loc.latitude
                
                #Getting the state and country
                coordinates=(latitude,longitude)
                h=(dict(rg.search(coordinates)[0]))
                country_code=h['cc']
                state=h['admin1'] if h['admin1']!='NCT' else 'New Delhi'
                
                #Sentiment analysis by vaderSentiment
                sentiment_tweet = sentiment.polarity_scores(text)
                tweet_compound=sentiment_tweet['compound']
                Overall_Sentiment='Neutral' if tweet_compound==0 else ('Positive' if tweet_compound>0 else 'Negative')
                
                with open('etweets.csv','a') as file:
                    writer=csv.DictWriter(file,fieldnames=column_headers)
                    new_tweet={
                        'User_Id':id_str,
                        'Created_at':created_at,
                        'Country_code':country_code,
                        'State':state,
                        'Latitude':latitude,
                        'Longitude':longitude,
                        'Tweet_text':text,
                        'Sentiment_compound':tweet_compound,
                        'Overall_sentiment':Overall_Sentiment
                    }
                    writer.writerow(new_tweet)
                
                print(created_at,"\n",text,"\n","user loc:",user_location)#,longitude,latitude)
                print(tweet_compound,Overall_Sentiment)
                print(country_code,state)
                
            except:
                return True
            
    def on_error(self, status_code):
        # Since Twitter API has rate limits, this stop srcraping data as it exceed to the thresold.
        if status_code == 420:
            # return False to disconnect the stream
            print("Error-")
            time.sleep(60 * 1)
            return 
        
    def on_exception(self, exception):
        print(exception)
        return
    
    def on_disconnect(self, notice):
        """Called when twitter sends a disconnect notice
        Disconnect codes are listed here:
        """
        print(notice)
        return

In [ ]:
auth=tweepy.OAuthHandler(API_key,API_secret_key)
auth.set_access_token(access_token,access_token_secret)
api=API(auth)

myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
while True:
    try:
        myStream.filter(languages=["en"], track = ['corona','covid-19','lockdown'])
                #locations=[68.1766451354, 7.96553477623, 97.4025614766, 35.4940095078])
    except:
        print("Error")
        continue

HTTPSConnectionPool(host='stream.twitter.com', port=443): Max retries exceeded with url: /1.1/statuses/filter.json?delimited=length (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')")))
Error
Loading formatted geocoded file...
2020-06-28 20:17:40 
 I don t think anything I ve seen so perfectly captures why there s no way the US is going to be getting on top of COVID 
 user loc: Glasgow, Scotland
0.7155 Positive
GB Scotland
2020-06-28 20:17:40 
 i tested positive for COVID 19 yesterday Something in my spirit is telling me to share my early signs symptoms andddd the h 
 user loc: South Central LA
0.7579 Positive
GB England
2020-06-28 20:17:40 
 I suspect this could be why Mark Sedwill s departure is making headlines today anything to take the spotlight from Cumm 
 user loc: North East, England
-0.296 Negative
GB England
2020-06-28 20:17:40 
 Ridiculous Unionist fury in advance despite social distancing when the funeral cortege arrived back in Belfast htt 
 u

2020-06-28 20:17:47 
 A shopkeepers goods are being destroyed he has paid for this goods Chinese company got their money Modi govt imports 
 user loc: Mumbai, India
-0.4939 Negative
IN Maharashtra
2020-06-28 20:17:47 
 I don t think anything I ve seen so perfectly captures why there s no way the US is going to be getting on top of COVID 
 user loc: Orlando, FL
0.7155 Positive
US Florida
2020-06-28 20:17:47 
 On my way home finally I ve been a nurse for more then 25 years and I m telling you I ve never ever in my life been throug 
 user loc: Seattle, WA
0.0 Neutral
US Washington
2020-06-28 20:17:47 
 Day 99 6 28 549197 inf 16487 deaths 110 gt 15 gt 11 gt 8 gt 6 Time period for addition of each lakh cases With shortage of labour 
 user loc: Patna, India
0.6597 Positive
IN Bihar
2020-06-28 20:17:47 
 I don t think anything I ve seen so perfectly captures why there s no way the US is going to be getting on top of COVID 
 user loc: the local olive garden 
0.7155 Positive
US Texas
2020-06-28